<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Asymmetric-Distance-computation" data-toc-modified-id="Asymmetric-Distance-computation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Asymmetric Distance computation</a></span><ul class="toc-item"><li><span><a href="#cython-version" data-toc-modified-id="cython-version-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>cython version</a></span></li></ul></li><li><span><a href="#Speed-up-precompute_adc-function" data-toc-modified-id="Speed-up-precompute_adc-function-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Speed up <code>precompute_adc</code> function</a></span></li></ul></div>

In [1]:
import numpy as np

In [2]:
%load_ext cython
import Cython

## Asymmetric Distance computation

Currently the code performs

```
dists = np.sum(self.dtable[range(M), codes], axis=1)
```
which is equivalent to 

```python
dists = np.zeros((N, )).astype(np.float32)
for n in range(N):
    for m in range(M):
        dists[n] += self.dtable[m][codes[n][m]]

```

In [3]:
M = 32
np.random.seed(123)

n_cluster = 256
dtable = np.array(np.random.random((M, n_cluster)), 'float32')

np.random.seed(123)
pq_codes_batch = np.array([np.random.randint([M]*M)])
N, M = pq_codes_batch.shape


In [4]:
pq_codes_batch

array([[30, 13, 30,  2, 28,  2,  6, 17, 19, 10, 27, 25, 22,  1,  0, 17,
        30, 15,  9,  0, 14,  0, 15, 25, 19, 14, 29,  4,  0, 16,  4, 17]])

In [5]:
dtable[range(M),pq_codes_batch].sum(axis=1)

array([17.402649], dtype=float32)

In [6]:
def distances_loop_py(N,M, dtable):
    dists = np.zeros((N, )).astype(np.float32)
    for n in range(N):
        for m in range(M):
            dists[n] += dtable[m, pq_codes_batch[n,m]]
    return dists

In [7]:
distances_loop_py(1,M,dtable)

array([17.402647], dtype=float32)

In [8]:
dtable.shape

(32, 256)

In [9]:
%timeit distances_loop_py(1,M,dtable)

12.8 µs ± 403 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [10]:
%timeit dtable[range(M),pq_codes_batch].sum(axis=1)

6.29 µs ± 106 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


There are more complicated cases where pq_codes_batch will produce

In [27]:
M = 32
np.random.seed(123)

n_cluster = 256
dtable = np.array(np.random.random((M, n_cluster, 4)), 'float32')

np.random.seed(123)
pq_codes_batch = np.array([np.random.randint([M]*M)])
N, M = pq_codes_batch.shape

dtable.shape

(32, 256, 4)

In [11]:
def distances_loop_py_3dtable(N,M, dtable):
    dists = np.zeros((N, )).astype(np.float32)
    for n in range(N):
        for m in range(M):
            dists[n] += dtable[m][pq_codes_batch[n][m]]
    return dists

        # dists = np.zeros((N, )).astype(np.float32)
        # for n in range(N):
        #     for m in range(M):
        #         dists[n] += self.dtable[m][codes[n][m]]

In [12]:
dtable[4][pq_codes_batch[0][4]]

0.82181364

### cython version

In [ ]:
%%cython
cimport numpy as cnp
cimport cython
             
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef dist_pqcode_to_codebook(long M, float[:,:] dtable, long[:] pq_code):
    cdef float dist = 0
    cdef int m
    
    for m in range(M):
        dist += dtable[m, pq_code[m]]

    return dist

In [ ]:
pq_code = pq_codes_batch.flatten()

In [ ]:
%timeit dist_pqcode_to_codebook(M, dtable, pq_code)

We can make the method work for generic types but this will have a penalty when called from python 

In [ ]:
%%cython
cimport numpy as cnp
cimport cython
from cython cimport integral, floating
             
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef floating distances_loop_cy2(integral M,
                                  floating[:,:] dtable,
                                  integral[:] pq_code):
    cdef floating dist = 0
    cdef integral m 
    
    for m in range(M):
        dist += dtable[m, pq_code[m]]

    return dist

In [ ]:
%timeit distances_loop_cy2(M, dtable, pq_code)

##### Batched version of dist_pqcodes_to_codebooks

In [ ]:
%%cython 

# distutils: language = c++

from libcpp.vector cimport vector

cimport numpy as np
cimport cython
import numpy as np

from cpython.array cimport array, clone

@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline float dist_pqcode_to_codebook(long M, float[:,:] dtable,long[:] pq_code):
    cdef float dist = 0
    cdef int m
    
    for m in range(M):
        dist += dtable[m, pq_code[m]]

    return dist

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef dist_pqcodes_to_codebooks(long M, float[:,:] dtable, long[:,:] pq_codes):
    cdef:
        int m, loops
        int N = pq_codes.shape[0] 
        float[:] dists = np.empty(N, dtype=np.float32)
    
    for n in range(N):
        dists[n] = dist_pqcode_to_codebook(M, dtable, pq_codes[n,:])

    return np.array(dists) 
             

In [ ]:
dist_pqcodes_to_codebooks(M, dtable, pq_codes_batch)

In [ ]:
%timeit dist_pqcodes_to_codebooks(M, dtable, pq_codes_batch)

Iin the batched problem we have the problem that we have to create a numpy array at runtime to store the distances. oreover we need to slice `pq_codes[n,:]` to call `dist_pqcode_to_codebook`.

We can improve this by instead of defining a numpy vector at runtime we can use a C++ vector and append the results.

In [ ]:
%%cython -a

# distutils: language = c++

from libcpp.vector cimport vector

cimport numpy as np
cimport cython
import numpy as np

from cpython.array cimport array, clone

@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline float dist_pqcode_to_codebook(long M, float[:,:] dtable,long[:] pq_code):
    cdef float dist = 0
    cdef int m
    
    for m in range(M):
        dist += dtable[m, pq_code[m]]

    return dist

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef dist_pqcodes_to_codebooks(long M, float[:,:] dtable, long[:,:] pq_codes):
    cdef:
        int m, loops
        int N = pq_codes.shape[0] 
        #float[:] dists = np.empty(N, dtype=np.float32)
        vector[float] dists
    
    for n in range(N):
        dists.push_back(dist_pqcode_to_codebook(M, dtable, pq_codes[n,:]))

    return np.asarray(dists)
             

In [ ]:
dist_pqcodes_to_codebooks(M, dtable, pq_codes_batch)

In [ ]:
%timeit dist_pqcodes_to_codebooks(M, dtable, pq_codes_batch)

In [ ]:
pq_codes_batch_50 = np.vstack([pq_codes_batch for i in range(50)])

In [ ]:
%timeit dist_pqcodes_to_codebooks(M, dtable, pq_codes_batch_50)

In [ ]:
%timeit dtable[range(M),pq_codes_batch_50].sum(axis=1)

If we don't need an output as a numpy array (a list is fine) we can go with the following implementation which is 2x faster

In [ ]:
%%cython

# distutils: language = c++

from libcpp.vector cimport vector

cimport numpy as np
cimport cython
import numpy as np

from cpython.array cimport array, clone

@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline float dist_pqcode_to_codebook(long M, float[:,:] dtable,long[:] pq_code):
    cdef float dist = 0
    cdef int m
    
    for m in range(M):
        dist += dtable[m, pq_code[m]]

    return dist

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef dist_pqcodes_to_codebooks(long M, float[:,:] dtable, long[:,:] pq_codes):
    cdef:
        int m, loops
        int N = pq_codes.shape[0] 
        #float[:] dists = np.empty(N, dtype=np.float32)
        vector[float] dists
    
    for n in range(N):
        dists.push_back(dist_pqcode_to_codebook(M, dtable, pq_codes[n,:]))

    return dists
             

In [ ]:
%timeit dist_pqcodes_to_codebooks(M, dtable, pq_codes_batch)

In [ ]:
%timeit dist_pqcodes_to_codebooks(M, dtable, pq_codes_batch_50)

## Speed up `precompute_adc` function

In [ ]:

import sklearn
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

from pqlite.core.codec.pq import PQCodec

D = 128
n_clusters = 256
M = 32

D = 128 # dimentionality / number of features
top_k = 100
n_cells = 18
Nt = 5000
d_subvector = int(D/M)
n_subvectors = M


np.random.seed(123)
Xtr, Xte =train_test_split(make_blobs(n_samples = Nt, n_features = D)[0].astype(np.float32), test_size=20)

pq = PQCodec(D, M, n_clusters)

In [ ]:
D, M, d_subvector, n_subvectors

In [ ]:
pq.fit(Xtr)

In [ ]:
codebooks = pq.codebooks
codebooks.shape

In [ ]:
codebooks[0].shape

In [ ]:
%timeit pq.precompute_adc(Xtr[4,:])

In [ ]:
query = Xtr[4,:]
#distance_table_from_class = pq.precompute_adc(query)
#distance_table_from_class.dtable.shape

In [ ]:
dtable = pq.precompute_adc(Xtr[4,:])
dtable.dtable.shape

In [ ]:

def precompute_adc3(query, 
                    d_subvector,
                    n_clusters,
                    codebooks):
    n_subvectors = int(D/d_subvector)
    dtable = np.empty((int(D/d_subvector), n_clusters), dtype=np.float32)

    for m in range(n_subvectors):
        query_sub = query[m * d_subvector : (m + 1) * d_subvector]
        print(query_sub)
        dtable[m, :] = np.linalg.norm(codebooks[m] - query_sub, axis=1) ** 2

    return dtable

dtable_custom = precompute_adc3(query, d_subvector, n_clusters, codebooks)
dtable_custom.shape

In [ ]:
np.testing.asserts_almost_equal(dtable_custom, dtable.dtable)

In [ ]:
codebooks.shape

In [ ]:
codebooks[0][]

We can avoid slices to use less memory and improve speed

In [ ]:
codebooks[0].shape

In [ ]:
codebooks.shape, M

In [ ]:
def precompute_adc3(query, 
                    d_subvector,
                    n_clusters,
                    codebooks):
    
    n_subvectors = int(D/d_subvector)
    dtable = np.empty((int(D/d_subvector), n_clusters), dtype=np.float32)
    query_subvec = np.empty(d_subvector, dtype=np.float32)
    query_subcodeword = np.empty(d_subvector, dtype=np.float32)
    
    for m in range(n_subvectors):
        
        # load m'th subquery
        i = 0
        for k in range(m * d_subvector, (m + 1) * d_subvector):
            query_subvec[i] = query[k]
            i += 1
            
        for ind_prototype in range(n_clusters):
            
            # load prototype ind_prototype for the m'th subspace
            for i in range(d_subvector):
                query_subcodeword[i] = codebooks[m, ind_prototype, i]
            
            # load prototype ind_prototype for the m'th subspace
            dist_subprototype_to_subquery = 0
            for k in range(d_subvector):
                coord_k = query_subcodeword[k] - query_subvec[k]
                dist_subprototype_to_subquery += coord_k * coord_k
            
            dtable[m, ind_prototype] = dist_subprototype_to_subquery
            
    return dtable

dtable_custom2 = precompute_adc3(query, d_subvector, n_clusters, codebooks)
dtable_custom2.shape

In [ ]:
#np.testing.assert_array_almost_equal(dtable_custom, dtable_custom2)
np.mean(dtable_custom - dtable_custom2)

We can cythonize the previous function

In [ ]:
%%cython -a 
cimport numpy as cnp
import numpy as np
cimport cython
from cython.cimports.libc.stdlib import malloc, free
from cpython.mem cimport PyMem_Malloc, PyMem_Realloc, PyMem_Free

             
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef float[:,:] precompute_adc_cy(float[:] query, 
                        long d_subvector,
                        long n_clusters,
                        float[::,:,:] codebooks):
            
    cdef int D = len(query)
    cdef int M = int(D/d_subvector) 
    cdef int n_subvectors = int(D/d_subvector)
    cdef int m,i,k,ind_prototype,j
    
    cdef float[::,:] dtable = np.empty((M, n_clusters), dtype=np.float32)
    cdef float[::] query_subvec = np.empty(d_subvector, dtype=np.float32)
    cdef float[::] query_subcodeword = np.empty(d_subvector, dtype=np.float32)
    
    #cdef double* query_subcodeword = <double*> PyMem_Malloc( d_subvector * sizeof(double))
        
    cdef float dist_subprototype_to_subquery, coord_j
    
    for m in range(n_subvectors):
        
        # load m'th subquery
        i = 0
        for k in range(m * d_subvector, (m + 1) * d_subvector):
            query_subvec[i] = query[k]
            i += 1
            
        for ind_prototype in range(n_clusters):
            
            # load prototype ind_prototype for the m'th subspace
            for i in range(d_subvector):
                query_subcodeword[i] = codebooks[m, ind_prototype, i]
            
            # load prototype ind_prototype for the m'th subspace
            dist_subprototype_to_subquery = 0.
            for j in range(d_subvector):
                coord_j = query_subcodeword[j] - query_subvec[j]
                dist_subprototype_to_subquery += coord_j * coord_j
            
            dtable[m, ind_prototype] = dist_subprototype_to_subquery
    
    free(query_subcodeword)
    
    return dtable


In [ ]:
%timeit dtable_custom3 = precompute_adc_cy(query, d_subvector, n_clusters, codebooks)

This is around a factor of 10x over the 'numpy vectorized' version

In [ ]:
%timeit pq.precompute_adc(Xtr[4,:])